<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2021_ex05a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2021-ex05a

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

# 2次元データのロジスティック回帰



正しく動作させるためには，関数 update の中身を修正する必要があります．それ以外は，いじらなくてもok．学習がちゃんと進むことが確認できたら，条件をいろいろ変えてみたらよいでしょう．

## 準備

In [ ]:
import numpy as np
import scipy as sp
#from scipy.misc import logsumexp     # for older SciPy 
from scipy.special import logsumexp
import matplotlib.pyplot as plt

##関数の定義

In [ ]:
### ロジスティック回帰モデル
#
def  model(X, W, b):
    
    # X, W, b が正しい shape かチェック
    assert X.ndim == 2   # (batchsize, D)
    assert W.ndim == 2  # (K, D) 
    assert b.ndim == 1   # (K,)
    K, D = W.shape
    assert X.shape[1] == D
    assert b.shape[0] == K
    
    # softmax の手前の値
    Y = X @ W.T + b[np.newaxis, :]  # (batchsize, K)
    
    # softmax （普通の計算法）
    #Ztmp = np.exp(Y)  # (batchsize, K)
    #Z = Ztmp / np.sum(Ztmp, axis = 1, keepdims = True)  # (batchsize, K)
    
    # softmax （logsumexp経由，こうする理由は後述）
    lse = logsumexp(Y, axis = 1, keepdims = True)  # (batchsize,1)
    Z = np.exp(Y - lse)                                                  # (batchsize, K)
    
    return Y, Z

In [ ]:
### 交差エントロピー （教師信号 t は 0 か 1 しかとらない前提で， t のかわりにクラスラベル lab を用いる）
#
def cross_entropy(Z, lab):
    
    # shape の整合性をチェック
    assert Z.ndim == 2      # (n, K)
    assert lab.ndim == 1   # (n,)
    n, K = Z.shape
    assert Z.shape[0] == n
    
    # 交差エントロピー
    Zk = np.choose(lab, Z.T) # Z の各行について， lab の値に該当する列の値のみ取り出す
    H = -np.log(Zk)
        
    return H

In [ ]:
### 交差エントロピー （logsumexp version）
#
def cross_entropy2(Y, lab):
    
    # shape の整合性をチェック
    assert Y.ndim == 2      # (n, K)
    assert lab.ndim == 1   # (n,)
    n, K = Y.shape
    assert Y.shape[0] == n

    # 交差エントロピー
    Yk = np.choose(lab, Y.T)                 # Y の各行について， lab の値に該当する列の値のみ取り出す
    H = - Yk + logsumexp(Y, axis = 1)  # こうやって計算する理由は後述
        
    return H

上記の関数たちの中で，softmax の計算の時に logsumexp を使ったり，交差エントロピーの計算の時に Z のかわりに Y を使ったりしているのは，その方が数値計算的に安全だから．もう少し詳しく知りたいひとは以下をどうぞ．
- http://takatakamanbou.hatenablog.com/entry/2015/03/26/214605
- http://takatakamanbou.hatenablog.com/entry/2015/01/23/235500
- https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.misc.logsumexp.html

In [ ]:
### パラメータの更新
#
def update(X, Z, Zt, W, b, eta):
    
    Zt_Z = Zt - Z
    
    # update W
    #dHdW = ##### ここを書きましょう #####
    W += eta * dHdW
    
    ### うまくいかないときは，Zt_Z.shape, X.shape, W.shape 等を print して考えてみるとよいかも
    
    # update b
    #dHdb  = ##### ここを書きましょう #####
    b += eta * dHdb

In [ ]:
### データ生成関数
#   2次元3クラス，N0, N1, N2 はクラスごとのサンプル数
#
def gendat3(N0, N1, N2):
    
    # クラス0
    mu0 = np.array([0.0, 0.0])
    cov0 = np.eye(2)
    dat0 = np.random.multivariate_normal(mu0, cov0, size = N0)  #  N0 x 2
    lab0 = np.zeros(N0, dtype = int)
    
    # クラス1
    mu1 = np.array([3.0, -4.0])
    cov1 = np.array([[0.875, 0.65],  [0.65, 1.625]])
    dat1 = np.random.multivariate_normal(mu1, cov1, size = N1)  #  N1 x 2
    lab1 = np.ones(N1, dtype = int)
                    
    # クラス2
    mu2 = np.array([4.0, 3.0])
    cov2 = np.array([[0.875, -0.65],  [-0.65, 1.625]])
    dat2 = np.random.multivariate_normal(mu2, cov2, size = N2)  #  N2 x 2
    lab2 = np.ones(N2, dtype = int) + 1
    
    dat = np.vstack((dat0, dat1, dat2))
    lab = np.hstack((lab0, lab1, lab2))
    
    return dat, lab

## データを生成して可視化

In [ ]:
# 学習データの生成
D = 2  # データの次元数
K = 3  # クラス数
datL, labL = gendat3(1000, 1000, 1000)
nL = datL.shape[0]

In [ ]:
# 学習データの散布図
for k in range(K):
    XX = datL[labL == k, :]
    plt.scatter(XX[:, 0], XX[:, 1], s = 10)

xmin, xmax = -8, 8
ymin, ymax = -8, 8

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
ax = plt.gca()
ax.set_aspect('equal')
plt.show()

## 学習させる

In [ ]:
# 教師信号の生成
ZtL = np.zeros((nL, K))
for k in range(K):
    ZtL[labL == k, k] = 1.0

# パラメータの初期化
W = 0.01 * np.random.random((K, D))
b = 0.01 * np.random.random(K)

# 実験条件の設定
nitr = 1000
eta = 0.01
batchsize = 64

# 学習の繰り返し
for i in range(nitr):
    
    # バッチの作成（ここでは毎回ランダムに選択）
    idx = np.random.randint(nL, size = batchsize)
    X = datL[idx, :]
    Zt = ZtL[idx, :]
    
    # 出力の計算とパタメータの更新
    Y, Z = model(X, W, b)
    update(X, Z, Zt, W, b, eta)
    
    # 適当なタイミングで交差エントロピーと識別率を表示
    if i < 10 or i % 50 == 0:
        Y, Z = model(datL, W, b)
        H = cross_entropy(Z, labL)
        #H = cross_entropy2(Y, labL)
        correct = np.sum(labL == np.argmax(Z, axis = 1))  # 正答数
        print('{0} {1:.5f} {2:.3f}'.format(i, np.mean(H), correct / nL))

## 事後確率を推定して可視化

nitr を小さくすれば学習が収束する前の様子を眺められる

In [ ]:
# 事後確率 p(y|x) の推定
x, y = np.mgrid[xmin:xmax:0.05, ymin:ymax:0.05]
X = np.dstack((x, y))
npoint  = X.shape[0]
X = np.vstack((x.reshape(-1), y.reshape(-1))).T
Y, Z = model(X, W, b)
P = Z.reshape((npoint, npoint, K))

# 各点で P の和が 1 になるよう正規化
P /= np.sum(P, axis = 2)[:, :, np.newaxis]

In [ ]:
# 事後確率の推定値を可視化

# 散布図ではクラス 0 が B, クラス 1 が G, クラス 2 が R  ==> 画素値は 0番が R, 1番が G, 2番が B 
imgP = np.empty_like(P)
imgP[:, :, 0] = P[:, :, 2]
imgP[:, :, 1] = P[:, :, 1]
imgP[:, :, 2] = P[:, :, 0]

# P は axis 0 が X軸， axis 1 が Y軸
imgP2 = np.transpose(imgP, axes = [1, 0, 2])
# 画像として表示する際は縦軸をひっくり返す必要あり
img = imgP2[::-1, :, :]
plt.imshow(img)